In [14]:
#Please see README for details regarding ComBat algorithm in general, where this code came from, and
#its uses and limitations.

#Please refer to sample runs for concrete examples of how each piece affects the normalization 
#(e.g., a case with no numerical covariates vs. one with, etc.).

#What follows are the essential pieces of the code to run the algorithm in python. 
#Resulting dataframe is identical for all values to dataframe produced in R using same same initial
#data (up to 1e(-5)).

In [15]:
#Packages/modules necessary
import pandas as pd
import numpy.linalg as la
import numpy as np
import patsy
import sys

In [16]:
#The following are functions that run within ComBat - each is briefly explained. 
#For details, see README.

In [17]:
#Design matrix based on identified batches and numerical covariates.
def design_mat(mod, numerical_covariates, batch_levels):
    # require levels to make sure they are in the same order as we use in the
    # rest of the script.
    design = patsy.dmatrix("~ 0 + C(batch, levels=%s)" % str(batch_levels),
                                                  mod, return_type="dataframe")

    mod = mod.drop(["batch"], axis=1)
    numerical_covariates = list(numerical_covariates)
    sys.stderr.write("found %i batches\n" % design.shape[1])
    other_cols = [c for i, c in enumerate(mod.columns)
                  if not i in numerical_covariates]
    factor_matrix = mod[other_cols]
    design = pd.concat((design, factor_matrix), axis=1)
    if numerical_covariates is not None:
        sys.stderr.write("found %i numerical covariates...\n"
                            % len(numerical_covariates))
        for i, nC in enumerate(numerical_covariates):
            cname = mod.columns[nC]
            sys.stderr.write("\t{0}\n".format(cname))
            design[cname] = mod[mod.columns[nC]]
    sys.stderr.write("found %i categorical variables:" % len(other_cols))
    sys.stderr.write("\t" + ", ".join(other_cols) + '\n')
    return design

In [18]:
#Hyperparameter used in delta_star calculation.
def aprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (2 * s2 +m**2) / np.float64(s2)

In [19]:
#Hyperparameter used in delta_star calculation.
def bprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (m*s2+m**3) / np.float64(s2)

In [20]:
#Used to calculate gamma_star - appears in it_sol and it_sol_loc functions.
def postmean(g_hat, g_bar, n, d_star, t2):
    return (t2*n*g_hat+d_star * g_bar) / (t2*n+d_star)

In [21]:
#Used to calculated delta_star - appears in it_sol and it_sol_scale functions.
def postvar(sum2, n, a, b):
    return (0.5 * sum2 + b) / (n / 2.0 + a - 1.0)

In [22]:
#If mean_only = False, used to generate gamma_star and delta_star
def it_sol(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        #print g_hat.shape, g_bar.shape, t2.shape
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)
       
        change = max((abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max())
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (g_new, d_new)
    return adjust

In [23]:
#If mean_only = True, used to generate gamma_star only
def it_sol_loc(g_hat, g_bar, n, d_hat, t2):
    n = 1
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    g_new = postmean(g_hat, g_bar, n, d_old, t2)
    g_old = g_new #.copy()
    adjust = (g_new)
    return adjust

In [24]:
#If mean_only = True, used to generate delta_star only
def it_sol_scale(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        #print g_hat.shape, g_bar.shape, t2.shape
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)
       
        change = (abs(d_new - d_old) / d_old).max()
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (d_new, g_new)
    return adjust

In [25]:
#The following is the actual ComBat function itself - each major section is briefly explained.
#See README for further details.

In [26]:
def combat(data, batch, model=None, numerical_covariates=None, mean_only=False, ref_batch=None):
    """Correct for batch effects in a dataset
    Parameters
    ----------
    data : pandas.DataFrame
        A (n_features, n_samples) dataframe of the expression to batch correct
    batch : List-like
        A column corresponding to the batches in the data, in the same order
        as the samples in ``data``
    model : patsy.design_info.DesignMatrix, optional
        A model matrix describing metadata on the samples which could be
        causing batch effects. If not provided, then will attempt to coarsely
        correct just from the information provided in ``batch``
    numerical_covariates : list-like
        List of covariates in the model which are numerical, rather than
        categorical
    mean_only : (T/F)
        Only adjusts for mean across the batches, forgoing scale (variance) adjustment. 
        Default = False
    ref_batch : int
        Adjusts all batches to the specified referance batch. Specified batch will not be adjusted.
        Default = None
        
    Returns
    -------
    corrected : pandas.DataFrame
        A (n_features, n_samples) dataframe of the batch-corrected data
    """
    #If designating numerical_covariates, this is the first point of incorporaiton.
    if isinstance(numerical_covariates, str):
        numerical_covariates = [numerical_covariates]
    
    if numerical_covariates is None:
        numerical_covariates = []

    if model is not None and isinstance(model, pd.DataFrame):
        model["batch"] = list(batch)
    
    else:
        model = pd.DataFrame({'batch': batch})
        
    #If using ref_batch, this is the first point of incorporation.
    if ref_batch is not None and isinstance(ref_batch, int):
        ref = int(ref_batch)

    batch_items = model.groupby("batch").groups.items()
    batch_levels = [k for k, v in batch_items]
    batch_info = [v for k, v in batch_items]
    n_batch = len(batch_info)
    n_batches = np.array([len(v) for v in batch_info])
    n_array = sum(n_batches)
    
    #Drop intercept
    drop_cols = [cname for cname, inter in  ((model == 1).all()).iteritems() if inter == True]
    drop_idxs = [list(model.columns).index(cdrop) for cdrop in drop_cols]
    model = model[[c for c in model.columns if not c in drop_cols]]
    numerical_covariates = [list(model.columns).index(c) if isinstance(c, str) else c
            for c in numerical_covariates if not c in drop_cols]
    
    #Creation of design matrix from design_mat function defined above
    design = design_mat(model, numerical_covariates, batch_levels)
    
    #If ref_batch designated, replaces 0s with 1s for all values in that column.
    if ref_batch is not None:
        design.iloc[:,ref-1] = 1.
    
    #Standardizes the data across genes.
    sys.stderr.write("Standardizing Data across genes.\n")
    B_hat = np.dot(np.dot(la.inv(np.dot(design.T, design)), design.T), data.T)
    
    #grand_mean and var_pooled calculated, depending on presence/absence of ref_batch designation.
    
    #NOTE: if using ref_batch, 'dft' (DataFrameTranspose) will be very important - it refers to 
    #whatever array contains your batch designating column, as well as any other columns specifying 
    #covariates. For example, in the bladderbatch data, this would be the array typically labeled
    #'pheno'.
    #The code below is set up for a dataframe that contains cell lines as index, 
    #gene names as columns, and a final column that contains batch number - thus making it easier
    # to pull out the cell lines and associated genetic info by batch. This is NOT the case
    #for something like the pheno' dataframe, which contains info corresponding to a seperate dataframe
    #that actually has the genetic data.
    #Depending on the way the 'dft' array is initially set up, an additional line of code and naming
    #modification may need to be added to line up the arrays, but it has purposly been kept as a 
    #simple, single extra line - see worked bladderbatch example with refbatch2.
    if ref_batch is not None:
        grand_mean = B_hat[ref-1]
        refdat = dft.loc[dft['batch'] == ref].drop('batch', axis=1)
        reflist=refdat.index.tolist()
        desbatch=design.T[reflist].T
        var_pooled = np.dot(((refdat - np.dot(desbatch, B_hat))**2).T, np.ones((n_batches[ref-1], 1)) / n_batches[ref-1])
            
    else:
        grand_mean = np.dot((n_batches / n_array).T, B_hat[:n_batch,:])
        var_pooled = np.dot(((data - np.dot(design, B_hat).T)**2), np.ones((n_array, 1)) / n_array)

    stand_mean = np.dot(grand_mean.T.reshape((len(grand_mean), 1)), np.ones((1, n_array)))
    tmp = np.array(design.copy())
    tmp[:,:n_batch] = 0
    stand_mean  += np.dot(tmp, B_hat).T
    
    #Standardized dataframe obtained.
    s_data = ((data - stand_mean) / np.dot(np.sqrt(var_pooled), np.ones((1, n_array))))

    #Get regression parameters - first point of mean_only argument incorporation.
    sys.stderr.write("Fitting L/S model and finding priors\n")
    batch_design = design[design.columns[:n_batch]]
    gamma_hat = np.dot(np.dot(la.inv(np.dot(batch_design.T, batch_design)), batch_design.T), s_data.T)

    #If mean_only = True, all delta_hat values set to 1.
    delta_hat = []

    for i, batch_idxs in enumerate(batch_info):
        if mean_only is False:
            delta_hat.append(s_data[batch_idxs].var(axis=1))
        
        else:
            x=s_data[batch_idxs].var(axis=1)
            x[x<100]=1
            np.float64(delta_hat.append(x))
            
    gamma_bar = gamma_hat.mean(axis=1) 
    t2 = gamma_hat.var(axis=1)
    
    #aprior and bprior functions defined above - create hyperparameters to be used in delta_star.
    a_prior = list(map(aprior, delta_hat))
    b_prior = list(map(bprior, delta_hat))
    
    sys.stderr.write("Finding parametric adjustments\n")
    
    #If adjusting for both location (mean) and scale (variance), will use it_sol function defined
    #above to generate gamma_star and delta_star for subsequent adjustment.
    
    #If mean_only = True, it_sol_loc function defined above will be used for gamma_star, and 
    #it_sol_scale will be used to first generate delta_star, and then delta_star 
    #values will be replaced by 1s.
    
    gamma_star, delta_star = [], []
    for i, batch_idxs in enumerate(batch_info):
    
        if mean_only is False:
            
            temp = it_sol(s_data[batch_idxs], gamma_hat[i],
            delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])

            gamma_star.append(temp[0])
            delta_star.append(temp[1])
        
        else:
            
            temploc = it_sol_loc(gamma_hat[i], gamma_bar[i], 1, delta_hat[i], t2[i])

            gamma_star.append(temploc)
        
            tempscale = it_sol_scale(s_data[batch_idxs], gamma_hat[i],
                     delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])
        
            y=tempscale[1]
            y[y<100]=1
            delta_star.append(y)
    
    #Normalize the data, depending on mean_only and ref_batch arguments incorporated above.
    sys.stdout.write("Adjusting data\n")
    bayesdata = s_data
    gamma_star = np.array(gamma_star)
    delta_star = np.array(delta_star)
    
    #Ensures ref_batch corresponding values are not adjusted.  
    if ref_batch is not None:
        gamma_star[ref-1]=0
        delta_star[ref-1]=1

    for j, batch_idxs in enumerate(batch_info):

        dsq = np.sqrt(delta_star[j,:])
        dsq = dsq.reshape((len(dsq), 1))
        denom =  np.dot(dsq, np.ones((1, n_batches[j])))
        numer = np.array(bayesdata[batch_idxs] - np.dot(batch_design.ix[batch_idxs], gamma_star).T)

        bayesdata[batch_idxs] = numer / denom
   
    vpsq = np.sqrt(var_pooled).reshape((len(var_pooled), 1))
    bayesdata = bayesdata * np.dot(vpsq, np.ones((1, n_array))) + stand_mean
   
    return bayesdata